# Kris Data

In [33]:
import pandas as pd;
import numpy as np;
import os;
import time;
import datetime;
import warnings;
warnings.filterwarnings('ignore');

In [34]:
path = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/03. Raw Data/Connect Data/Kris Data"
files = os.listdir(path)

Selecting only **excel files** from all the files available in the above path

In [35]:
files_xls = [f for f in files if (f[:8] == "Kris LDL")]
#files_xls = [f for f in files if (f[-8:] == "PTB.xlsx")|(f == "Kris Batt Hybrid.xlsx")]
#files_xls = [f for f in files if (f == "Kris Pour.xlsx") | (f == "Kris APC+WW and Abrasive.xlsx")]
#files_xls = [f for f in files if( f[:7] == "TotalFS" )| (f[:6] == 'Sheets')|( f[:5] == "LiqFS" )| (f[:6] == 'ScentB')| (f[:5] == 'Beads')] 
print("Total number of files in the folder are ",len(files_xls))

Total number of files in the folder are  1


In [36]:
files_xls

['Kris LDL.xlsx']

**Appending the read files in rows and storing them in a single dataframe** named ***UPC_Monthly***.

Printing number of columns and number of facts in each file.
Also checking the execution time of the code

In [37]:
time_start= time.time()

Kris_Data= pd.DataFrame()

j=0
for f in files_xls:
    
    data = pd.read_excel(path+"/"+f)
    print("\033[1m" + files_xls[j] + "\033[0m",":->\n# of Facts are ",len(set(data["Periods"])),"\t\t # of columns are ",len(data.columns))  
    Kris_Data = pd.concat([Kris_Data,data],axis=0,ignore_index=True)
    j=j+1
    
time_end= time.time();

print("execution time is ", time_end-time_start)

Kris LDL.xlsx :->
# of Facts are  14 		 # of columns are  12
execution time is  3.1859920024871826


In [38]:
# Relaceing in Periods(Facts)
Kris_Data["Periods"]=Kris_Data["Periods"].str.replace("$","temp")
Kris_Data["Periods"]=Kris_Data["Periods"].str.replace("- temp -","-")
Kris_Data["Periods"]=Kris_Data["Periods"].str.replace("temp","$")


Kris_Data["Periods"]=Kris_Data["Periods"].str.replace("CP FORM","CP CATEGORY")
Kris_Data["Periods"]=Kris_Data["Periods"].str.replace("CP USE","CP CATEGORY")
Kris_Data["Periods"]=Kris_Data["Periods"].str.replace("CP LDL AND FOAM","CP CATEGORY")

Kris_Data["Periods"]=Kris_Data["Periods"].str.replace("CP LDL AND FM X DISHAIDS/WPS/SPR","CP CATEGORY")

Kris_Data["Periods"]=Kris_Data["Periods"].str.replace("CP MEGA CATEGORY","CP CATEGORY")
Kris_Data["Periods"]=Kris_Data["Periods"].str.replace("CP BATTERY AND HYBRID","CP CATEGORY")


# Market name replaced
Kris_Data["Markets"]=Kris_Data["Markets"].str.replace("Walgreens Corp Total TA","Walgreens Total TA")

#replacing NaN values with 0
Kris_Data.fillna(value=0,inplace=True)



In [39]:
##Removing unwanted characters from all the column names** and store them in a list ***new_col_names***

new_col_names=[]
for i in Kris_Data.columns:
    x=len(i)
    if x<=17:
        new_col_names.append(i)
    elif i[:3]=='YTD':
        new_col_names.append(i[:-16].strip('- '))
    else:
        new_col_names.append(i[:-14].strip('- '))
        

## Checking the duplicacte column in** ***Kris_Data***

if(len(new_col_names)!=len(set(new_col_names))):
    print("\033[1;31m"+"*******************************ERROR*******************************\n\t\t\tWe have duplicate columns\n\t\t\tPlease recheck raw data files\n*******************************ERROR*******************************")

##Renaming the column names of ***Kris_Data***

# Kris_Data.rename(columns=dict(zip(Kris_Data.columns,new_col_names)),inplace=True)
Kris_Data.columns=new_col_names

In [40]:
new_col_names

['Markets',
 'Products',
 'Periods',
 'Latest 13 Wks',
 'Latest 26 Wks',
 'Cal Yr 2020',
 'Cal Yr 2019',
 'Cal Yr 2018',
 'Cal Yr 2017',
 'YTD',
 'YTD YA',
 'Feb 21']

In [41]:
##**Adding Missing Quater**


# GETTING 20 DAYS BEFORE MONTHS AND YEAR VALUES
now=datetime.datetime.now()
delay= datetime.timedelta(days=20)
a= now - delay
month_number=a.month
year=str(a.year-2)[-2:]
print(month_number)
print(year)

if( month_number >= 1 & month_number <= 3):
    Kris_Data['AMJ '+year]=""
    Kris_Data['JAS '+year]=""
    Kris_Data['OND '+year]=""
elif ( month_number >= 4 & month_number <= 6 ):
    Kris_Data['JAS '+year]=""
    Kris_Data['OND '+year]=""
elif ( month_number >= 7 & month_number <= 9  ):
    Kris_Data['OND '+year]=""

Kris_Data.shape

2
19


(10850, 15)

In [42]:
Kris_Data.columns

Index(['Markets', 'Products', 'Periods', 'Latest 13 Wks', 'Latest 26 Wks',
       'Cal Yr 2020', 'Cal Yr 2019', 'Cal Yr 2018', 'Cal Yr 2017', 'YTD',
       'YTD YA', 'Feb 21', 'AMJ 19', 'JAS 19', 'OND 19'],
      dtype='object')

In [43]:
##**Time period mapping**



time_period_map= pd.DataFrame({'Connect Time Period':['Markets', 'Products', 'Periods','Cal Yr 2020','Cal Yr 2019','Cal Yr 2018','Cal Yr 2017','YTD','YTD YA','Latest 13 Wks','Latest 26 Wks','OND 21','JAS 21','AMJ 21','JFM 21','OND 20','JAS 20','AMJ 20','JFM 20','OND 19','JAS 19','AMJ 19','Dec 21','Nov 21','Oct 21','Sep 21','Aug 21','Jul 21','Jun 21','May 21','Apr 21','Mar 21','Feb 21','Jan 21','Dec 20','Nov 20','Oct 20','Sep 20','Aug 20','Jul 20','Jun 20','May 20','Apr 20','Mar 20','Feb 20','Jan 20'],
                               'Final Name':['Markets', 'Products', 'Periods','FY 2020','FY 2019','FY 2018','FY 2017','YTD 2021','YTD 2020','Latest 13 Weeks','Latest 26 Weeks','Q4 2021','Q3 2021','Q2 2021','Q1 2021','Q4 2020','Q3 2020','Q2 2020','Q1 2020','Q4 2019','Q3 2019','Q2 2019','Dec 21','Nov 21','Oct 21','Sep 21','Aug 21','Jul 21','Jun 21','May 21','Apr 21','Mar 21','Feb 21','Jan 21','Dec 20','Nov 20','Oct 20','Sep 20','Aug 20','Jul 20','Jun 20','May 20','Apr 20','Mar 20','Feb 20','Jan 20']
                              ,'Order':range(1,47)})

time_period_map.head(5)

##We need to **change column names** of ***Kris_Data*** and **reorder** them based on the time_period_mapping

# assigning variables
LM_column= list(Kris_Data.columns)
time_connect=list(time_period_map["Connect Time Period"])
time_final=list(time_period_map["Final Name"])

# changing column names based on time_period_mapping
new_column=[]
for i in range(0,len(LM_column)):
    for j in range(0,len(time_connect)):
        if LM_column[i]==time_connect[j]:
            new_column.append(time_final[j])
        

print(new_column)

print(len(new_column),len(Kris_Data.columns))

Kris_Data.columns

#assigning new column names
Kris_Data.columns = new_column

# reordering column names
my_list =time_period_map.iloc[:, 1]
my_list1=Kris_Data.columns
result = [x for x in my_list if x in my_list1]
Kris_Data=Kris_Data[result]

['Markets', 'Products', 'Periods', 'Latest 13 Weeks', 'Latest 26 Weeks', 'FY 2020', 'FY 2019', 'FY 2018', 'FY 2017', 'YTD 2021', 'YTD 2020', 'Feb 21', 'Q2 2019', 'Q3 2019', 'Q4 2019']
15 15


In [44]:
##**Period Mapping**

Time_Mapping={
    "Nov 19":"4 W/E 11/23/19",
    "Dec 19":"5 W/E 12/28/19",
    "Jan 20":"4 W/E 02/01/20",
    "Feb 20":"4 W/E 02/29/20",
    "Mar 20":"5 W/E 04/04/20",
    "Apr 20":"4 W/E 05/02/20",
    "May 20":"4 W/E 05/30/20",
    "Jun 20":"5 W/E 07/04/20",
    "Jul 20":"4 W/E 08/01/20",
    "Aug 20":"4 W/E 08/29/20",
    "Sep 20":"5 W/E 10/03/20",
    "Oct 20":"4 W/E 10/31/20",
    "Nov 20":"4 W/E 11/28/20",
    "Dec 20":"5 W/E 01/02/21",
    "Jan 21":"4 W/E 01/30/21",
    "Feb 21":"4 W/E 02/27/21",
    "Mar 21":"5 W/E 04/03/21",
    "Apr 21":"4 W/E 05/01/21",
    "May 21":"4 W/E 05/29/21",
    "Jun 21":"5 W/E 07/03/21",
    "Jul 21":"4 W/E 07/31/21",
    "Aug 21":"4 W/E 08/28/21",
    "Sep 21":"5 W/E 10/02/21",
    "Oct 21":"4 W/E 10/30/21",
    "Nov 21":"4 W/E 11/27/21",
    "Dec 21":"5 W/E 01/01/22",
    "Jan 22":"4 W/E 01/29/22",
    "Feb 22":"4 W/E 02/26/22",
    "Mar 22":"5 W/E 04/02/22",
    "Apr 22":"4 W/E 04/30/22",
    "May 22":"5 W/E 05/28/22",
    "Jun 22":"4 W/E 06/25/22",
    "Jul 22":"4 W/E 07/23/22",
    "Aug 22":"5 W/E 08/20/22",
    "Sep 22":"4 W/E 09/17/22",
    "Oct 22":"4 W/E 10/15/22",
    "Nov 22":"5 W/E 11/12/22",
    "Dec 22":"4 W/E 12/10/22",
    "Jan 23":"4 W/E 01/07/23",
    "Feb 23":"5 W/E 02/04/23",
    "Mar 23":"4 W/E 03/04/23",
    "Apr 23":"4 W/E 04/01/23",
    "May 23":"5 W/E 04/29/23",
    "Jun 23":"4 W/E 05/27/23",
    "Jul 23":"4 W/E 06/24/23",
    "Aug 23":"5 W/E 07/22/23",
    "Sep 23":"4 W/E 08/19/23",
    "Oct 23":"5 W/E 09/16/23",
    "Nov 23":"4 W/E 10/14/23",
    "Dec 23":"4 W/E 11/11/23",
    "Jan 24":"5 W/E 12/09/23",
    "Feb 24":"4 W/E 01/06/24",
    "Mar 24":"4 W/E 02/03/24",
    "Apr 24":"5 W/E 03/02/24",
    "May 24":"4 W/E 03/30/24",
    "Jun 24":"4 W/E 04/27/24",
    "Jul 24":"5 W/E 05/25/24",
    "Aug 24":"4 W/E 06/22/24",
    "Sep 24":"4 W/E 07/20/24",
    "Oct 24":"5 W/E 08/17/24",
    "Nov 24":"4 W/E 09/14/24",
    "Dec 24":"4 W/E 10/12/24",
    "Jan 25":"5 W/E 11/09/24",
    "Feb 25":"4 W/E 12/07/24",
    "Mar 25":"5 W/E 01/04/25",
    "Apr 25":"4 W/E 02/01/25",
    "May 25":"4 W/E 03/01/25",
    "Jun 25":"5 W/E 03/29/25",
    "Jul 25":"4 W/E 04/26/25",
    "Aug 25":"4 W/E 05/24/25",
    "Sep 25":"5 W/E 06/21/25",
    "Oct 25":"4 W/E 07/19/25",
    "Nov 25":"4 W/E 08/16/25",
    "Dec 25":"5 W/E 09/13/25",
    "Jan 26":"4 W/E 10/11/25",
    "Feb 26":"4 W/E 11/08/25",
    "Mar 26":"5 W/E 12/06/25",
    "Apr 26":"4 W/E 01/03/26",
    "May 26":"4 W/E 01/31/26",
    "Jun 26":"5 W/E 02/28/26",
    "Jul 26":"4 W/E 03/28/26",
    "Aug 26":"5 W/E 04/25/26",
    "Sep 26":"4 W/E 05/23/26",
    "Oct 26":"4 W/E 06/20/26",
    "Nov 26":"5 W/E 07/18/26",
    "Dec 26":"4 W/E 08/15/26"
}


Kris_Data.rename(columns=Time_Mapping,inplace=True)

In [45]:
Market_Mapping={
    
     'ALBSCO Total Company TA':'Alb/Sfy',
     'Agency Grocery':'Agency Grocery',
     'Ahold USA Corp Total TA':"Ahold",
     'BJs Total TA':"BJs",
     'CLUB':"CLUB",
     'CVS Total Corp WO HI TA':"CVS",
     'Dol Gen Total TA':"Dollar General",
     'El Super Total TA':'El Super',
     'FDM':"FDM",
     'Family Dollar Total TA':"Family Dollar",
     'Fiesta Total TA':"Fiesta",
     'Giant Eagle Total TA':"Giant Eagle",
     'Kmart Total TA':"Kmart",
     'Meijer Total TA':"Meijer",
     'Publix Total TA':"Publix",
     'Rite Aid - Corp Total TA':"Rite Aid",
     "Sam's Total TA":"Sams",
     'ShopRite Total TA':"Shoprite",
     'Strategic Grocery':"Strategic Grocery",
     'TOTAL MASS':"TOTAL MASS",
     'Target Total BM/FF TA':"Target",
     'Total US Drug':"US Drug",
     'Total US Food':"US Food",
     'Total US xAOC':"US AOC",
     'VDC':"VDC",
     'Walgreens Total TA':"Walgreens",
     'Walmart Total US TA':"Walmart",
     'Wegmans Total TA':"Wegmans"
    }


Kris_Data["Markets"].replace(Market_Mapping,inplace=True)

In [46]:
#Changing Column Name

Kris_Data.rename(columns={"Periods":"Facts"},inplace=True)


In [47]:
##**Filter Condition**
#Taking only those products whose **Latest Month Dollar Share** is **greater than 0.07** for all the markets

# Kris_Data.iloc[:,3:]= Kris_Data.iloc[:,3:].apply(pd.to_numeric)

# Kris_Data.info()

# df=Kris_Data[(Kris_Data['Facts'] == '$ Shr - CP CATEGORY') & ( Kris_Data.iloc[:,19] > 0.07 )]

# # total number of unique Products 
# len(np.unique(list(df["Products"])))

# Kris_Data=Kris_Data[Kris_Data['Products'].isin(list(df["Products"]))]
# Kris_Data.head(3)

# len(set(Kris_Data["Products"]))

In [48]:
#***movecol*** function to **rearranging** the columns of a dataframe

def movecol(df, cols_to_move=[], ref_col='', place=''):
    cols = df.columns.tolist()
    if place == 'After':
        seg1 = cols[:list(cols).index(ref_col) + 1]
        seg2 = cols_to_move
    if place == 'Before':
        seg1 = cols[:list(cols).index(ref_col)]
        seg2 = cols_to_move + [ref_col]
    seg1 = [i for i in seg1 if i not in seg2]
    seg3 = [i for i in cols if i not in seg1 + seg2]
    
    return(df[seg1 + seg2 + seg3])

In [49]:
Kris_Data.head()

,Markets,Products,Facts,FY 2020,FY 2019,FY 2018,FY 2017,YTD 2021,YTD 2020,Latest 13 Weeks,Latest 26 Weeks,Q4 2019,Q3 2019,Q2 2019,4 W/E 02/27/21
0,US AOC,LDL/DISHAIDS/WPS/FM/SPRAYS | COLGATE-PALMOLIVE...,$ Shr - CP CATEGORY,0.000,0.001,0.024,0.084,0.0,0.000,0.0,0.0,,,,0.0
1,US AOC,LDL/DISHAIDS/WPS/FM/SPRAYS | COLGATE-PALMOLIVE...,$ Shr Chg YA - CP CATEGORY,-0.001,-0.023,-0.060,0.084,0.0,-0.003,0.0,0.0,,,,0.0
2,US AOC,LDL/DISHAIDS/WPS/FM/SPRAYS | COLGATE-PALMOLIVE...,Base $ Shr - CP CATEGORY,0.000,0.001,0.021,0.078,0.0,0.000,0.0,0.0,,,,0.0
3,US AOC,LDL/DISHAIDS/WPS/FM/SPRAYS | COLGATE-PALMOLIVE...,Base $ Shr Chg YA - CP CATEGORY,-0.001,-0.020,-0.057,0.078,0.0,-0.005,0.0,0.0,,,,0.0
4,US AOC,LDL/DISHAIDS/WPS/FM/SPRAYS | COLGATE-PALMOLIVE...,%ACV,0.000,0.022,0.387,0.731,0.0,0.000,0.0,0.0,,,,0.0


In [50]:
Kris_Data[["ShareBasis","Manufacturer","Franchise","Brand","SubBrand","UPC","Item"]] = Kris_Data['Products'].str.split('|',n=6,expand=True)


Kris_Data[["ShareBasis","Manufacturer","Franchise","Brand","SubBrand","UPC","Item"]]=Kris_Data[["ShareBasis","Manufacturer","Franchise","Brand","SubBrand","UPC","Item"]].fillna(value="")
Kris_Data.head(3)

# Moving columns after "Market"
Kris_Data = movecol(Kris_Data, 
             cols_to_move=["ShareBasis","Manufacturer","Franchise","Brand","SubBrand","UPC","Item"], 
             ref_col='Markets',
             place='After')
Kris_Data.head(3)

,Markets,ShareBasis,Manufacturer,Franchise,Brand,SubBrand,UPC,Item,Products,Facts,...,FY 2018,FY 2017,YTD 2021,YTD 2020,Latest 13 Weeks,Latest 26 Weeks,Q4 2019,Q3 2019,Q2 2019,4 W/E 02/27/21
0,US AOC,LDL/DISHAIDS/WPS/FM/SPRAYS,COLGATE-PALMOLIVE,AJAX,AJAX,AJAX BLEACH ALTERNATIVE,003500044499,AJAX DISHWASHING DETERGENT LIQUID CITRUS BERR...,LDL/DISHAIDS/WPS/FM/SPRAYS | COLGATE-PALMOLIVE...,$ Shr - CP CATEGORY,...,0.024,0.084,0.0,0.000,0.0,0.0,,,,0.0
1,US AOC,LDL/DISHAIDS/WPS/FM/SPRAYS,COLGATE-PALMOLIVE,AJAX,AJAX,AJAX BLEACH ALTERNATIVE,003500044499,AJAX DISHWASHING DETERGENT LIQUID CITRUS BERR...,LDL/DISHAIDS/WPS/FM/SPRAYS | COLGATE-PALMOLIVE...,$ Shr Chg YA - CP CATEGORY,...,-0.060,0.084,0.0,-0.003,0.0,0.0,,,,0.0
2,US AOC,LDL/DISHAIDS/WPS/FM/SPRAYS,COLGATE-PALMOLIVE,AJAX,AJAX,AJAX BLEACH ALTERNATIVE,003500044499,AJAX DISHWASHING DETERGENT LIQUID CITRUS BERR...,LDL/DISHAIDS/WPS/FM/SPRAYS | COLGATE-PALMOLIVE...,Base $ Shr - CP CATEGORY,...,0.021,0.078,0.0,0.000,0.0,0.0,,,,0.0


In [51]:
cols=list(Kris_Data.columns[0:9])
Kris_Data[cols]=Kris_Data[cols].apply(lambda x : x.str.strip())

**Selection**

In [52]:
Kris_Facts_Mapping={
    '$':"Dollars",
     '$ % Chg YA':"Dollars_Chg",
     '$ / $MM ACV':"Velocity",
     '$ / $MM ACV Chg YA':"Velocity_Chg",
     '$ Shr - CP CATEGORY':"Doll_Shr",
     '$ Shr Chg YA - CP CATEGORY':"Doll_Shr_Chg",
     '%ACV':"Store_selling",
     '%ACV Chg YA':"Store_selling_Chg",
     'Base $ Shr - CP CATEGORY':"Base_Doll_Shr",
     'Base $ Shr Chg YA - CP CATEGORY':"Base_Doll_Shr_Chg",
     'TDP':"Cum_Distri_Pts",
     'TDP Chg YA':"Cum_Distri_Pts_Chg",
     'Units':"Units",
     'Units / $MM ACV':"Units_Velocity"
    }

Kris_Data["Facts"]=Kris_Data["Facts"].replace(Kris_Facts_Mapping)

In [53]:
set(Kris_Data["ShareBasis"])

{'LDL/DISHAIDS/WPS/FM/SPRAYS'}

## Toothpaste

In [ ]:
Kris_TP=Kris_Data[Kris_Data["ShareBasis"]=="TOOTHPASTE"]

In [ ]:
# Facts=["Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg","Prcnt_ACV","Units","Units_Velocity"]
# Markets=[]

In [ ]:
Kris_TP.info()

In [ ]:
Time_Period=list(Kris_TP.columns[17:18])+list(Kris_TP.columns[12:14])+list(Kris_TP.columns[11:12])+list(Kris_TP.columns[10:11])
print(Time_Period)

In [ ]:
Kris_TP.columns

In [ ]:
Kris_TP.rename(columns={'Markets':"MKT", 'Item':"PROD", 'Manufacturer':"CP.MANUFACTURER",
                           'Franchise':"CP.FRANCHISE", 'Brand':"CP.BRAND",
                           'SubBrand':"CP.SUBBRAND", 'UPC':"PRODUCT.CODE",
                           },inplace=True)

In [ ]:
df1=pd.melt(Kris_TP, id_vars =["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
df2["Prcnt_ACV"]=df2["Store_selling"]

final_column_sorting=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg",
                      "Prcnt_ACV","Units","Units_Velocity"
                      ]
Kris_Data_final=df2[final_column_sorting]

In [ ]:
Kris_Data_final.head(3)

In [ ]:
Kris_Data_final.to_csv(path_TP+"kris_data_TP.csv",index=False)

## MW

In [ ]:
Kris_MW=Kris_Data[Kris_Data["ShareBasis"]=="MOUTHWASH"]

In [ ]:
# Facts=["Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg","Prcnt_ACV","Units","Units_Velocity"]
# Markets=[]

In [ ]:
Kris_MW.info()

In [ ]:
Time_Period=list(Kris_MW.columns[21:22])+list(Kris_MW.columns[16:18])+list(Kris_MW.columns[14:15])+list(Kris_MW.columns[10:11])
print(Time_Period)

In [ ]:
Kris_MW.columns

In [ ]:
Kris_MW.rename(columns={'Markets':"MKT", 'Item':"PROD", 'Manufacturer':"CP.MANUFACTURER",
                           'Franchise':"CP.FRANCHISE", 'Brand':"CP.BRAND",
                           'SubBrand':"CP.SUBBRAND", 'UPC':"PRODUCT.CODE",
                           },inplace=True)

In [ ]:
df1=pd.melt(Kris_MW, id_vars =["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
df2["Prcnt_ACV"]=df2["Store_selling"]

final_column_sorting=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg",
                      "Prcnt_ACV","Units","Units_Velocity"
                      ]
Kris_Data_final=df2[final_column_sorting]

In [ ]:
Kris_Data_final.head(3)

In [ ]:
path_MW = r"//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/MW"

In [ ]:
Kris_Data_final.to_csv(path_MW+"/kris_data_MW.csv",index=False)

## MTB

In [ ]:
path_MTB = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/MTB/"

In [ ]:
Kris_MTB=Kris_Data[Kris_Data["ShareBasis"]=="MANUAL"]

In [ ]:
# Facts=["Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg","Prcnt_ACV","Units","Units_Velocity"]
# Markets=[]

In [ ]:
Kris_MTB.info()

In [ ]:
Time_Period=list(Kris_MTB.columns[19:20])+list(Kris_MTB.columns[14:16])+list(Kris_MTB.columns[12:13])+list(Kris_MTB.columns[10:11])
print(Time_Period)

In [ ]:
Kris_MTB.columns

In [ ]:
Kris_MTB.rename(columns={'Markets':"MKT", 'Item':"PROD", 'Manufacturer':"CP.MANUFACTURER",
                           'Franchise':"CP.FRANCHISE", 'Brand':"CP.BRAND","ShareBasis":"CP_SHARE_BASIS",
                           'SubBrand':"CP.SUBBRAND", 'UPC':"PRODUCT.CODE",
                           },inplace=True)

In [ ]:
df1=pd.melt(Kris_MTB, id_vars =["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","Facts","CP_SHARE_BASIS"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE",'PER',"CP_SHARE_BASIS"],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
df2["Prcnt_ACV"]=df2["Store_selling"]

final_column_sorting=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg",
                      "Prcnt_ACV","Units","Units_Velocity","CP_SHARE_BASIS"
                      ]
Kris_Data_final=df2[final_column_sorting]

In [ ]:
Kris_Data_final.head(3)

In [ ]:
Kris_Data_final.to_csv(path_MTB+"kris_data_MTB.csv",index=False)

### PTB

In [ ]:
Kris_PTB=Kris_Data[Kris_Data["ShareBasis"].isin(['BATTERY+HYBRID', 'PORTABLE TB', 'POWER'])]

In [ ]:
Kris_PTB.shape

In [ ]:
Kris_PTB["ShareBasis"].replace({"POWER":"TOTAL POWER","PORTABLE TB":"TOTAL POWER"},inplace = True)

In [ ]:
Kris_PTB.shape

In [ ]:
set(Kris_PTB["ShareBasis"])

In [ ]:
# Facts=["Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg","Prcnt_ACV","Units","Units_Velocity"]
# Markets=[]

In [ ]:
Kris_PTB.info()

In [ ]:
Time_Period=list(Kris_PTB.columns[19:20])+list(Kris_PTB.columns[14:16])+list(Kris_PTB.columns[12:13])+list(Kris_PTB.columns[10:11])
print(Time_Period)

In [ ]:
Kris_PTB.columns

In [ ]:
Kris_PTB.rename(columns={'Markets':"MKT", 'Item':"PROD", 'Manufacturer':"CP.MANUFACTURER",
                           'Franchise':"CP.FRANCHISE", 'Brand':"CP.BRAND","ShareBasis":"CP_SHARE_BASIS",
                           'SubBrand':"CP.SUBBRAND", 'UPC':"PRODUCT.CODE",
                           },inplace=True)

In [ ]:
df1=pd.melt(Kris_PTB, id_vars =["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","Facts","CP_SHARE_BASIS"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE",'PER',"CP_SHARE_BASIS"],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['CP_BATTERY_HYBRID','MKT','PROD','PER'],inplace=True)

In [ ]:
df2["Prcnt_ACV"]=df2["Store_selling"]

final_column_sorting=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg",
                      "Prcnt_ACV","Units","Units_Velocity","CP_SHARE_BASIS"
                      ]
Kris_Data_final=df2[final_column_sorting]

In [ ]:
Kris_Data_final.head(15)

In [ ]:
Kris_PTB = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/PTB/"

Kris_Data_final.to_csv(Kris_PTB+"kris_data_PTB.csv",index=False)

## HHC

In [ ]:
path_HHC = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/HHC/"

In [ ]:
Kris_HHC=Kris_Data[Kris_Data["ShareBasis"].isin(['ABRASIVE', 'APC+WW', 'POUR APC+WW'])]

In [ ]:
# Facts=["Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg","Prcnt_ACV","Units","Units_Velocity"]
# Markets=[]

In [ ]:
Kris_HHC.info()

In [ ]:
Time_Period=list(Kris_HHC.columns[21:22])+list(Kris_HHC.columns[16:18])+list(Kris_HHC.columns[14:15])+list(Kris_HHC.columns[10:11])
print(Time_Period)

In [ ]:
Kris_HHC.columns

In [ ]:
Kris_HHC.rename(columns={'Markets':"MKT", 'Item':"PROD", 'Manufacturer':"CP.MANUFACTURER",
                           'Franchise':"CP.FRANCHISE", 'Brand':"CP.BRAND","ShareBasis":"CP_SHARE_BASIS",
                           'SubBrand':"CP.SUBBRAND", 'UPC':"PRODUCT.CODE",
                           },inplace=True)

In [ ]:
df1=pd.melt(Kris_HHC, id_vars =["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","Facts","CP_SHARE_BASIS"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE",'PER',"CP_SHARE_BASIS"],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
df2["Prcnt_ACV"]=df2["Store_selling"]

final_column_sorting=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg",
                      "Prcnt_ACV","Units","Units_Velocity","CP_SHARE_BASIS"
                      ]
Kris_Data_final=df2[final_column_sorting]

In [ ]:
Kris_Data_final.head(3)

In [ ]:
del Kris_Data_final["CP_SHARE_BASIS"]

In [ ]:
Kris_Data_final.to_csv(path_HHC+"kris_data_HHC.csv",index=False)

## LHS


In [21]:
Kris_LHS=Kris_Data[Kris_Data["ShareBasis"]=="LIQUID"]

In [22]:
# Facts=["Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg","Prcnt_ACV","Units","Units_Velocity"]
# Markets=[]

In [23]:
Kris_LHS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8526 entries, 0 to 8525
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Markets          8526 non-null   object 
 1   ShareBasis       8526 non-null   object 
 2   Manufacturer     8526 non-null   object 
 3   Franchise        8526 non-null   object 
 4   Brand            8526 non-null   object 
 5   SubBrand         8526 non-null   object 
 6   UPC              8526 non-null   object 
 7   Item             8526 non-null   object 
 8   Products         8526 non-null   object 
 9   Facts            8526 non-null   object 
 10  FY 2020          8526 non-null   float64
 11  FY 2019          8526 non-null   float64
 12  YTD 2021         8526 non-null   float64
 13  YTD 2020         8526 non-null   float64
 14  Latest 13 Weeks  8526 non-null   float64
 15  Latest 26 Weeks  8526 non-null   float64
 16  Q4 2019          8526 non-null   object 
 17  Q3 2019       

In [24]:
Time_Period=list(Kris_LHS.columns[19:20])+list(Kris_LHS.columns[14:16])+list(Kris_LHS.columns[12:13])+list(Kris_LHS.columns[10:11])
print(Time_Period)

['4 W/E 02/27/21', 'Latest 13 Weeks', 'Latest 26 Weeks', 'YTD 2021', 'FY 2020']


In [25]:
Kris_LHS.columns

Index(['Markets', 'ShareBasis', 'Manufacturer', 'Franchise', 'Brand',
       'SubBrand', 'UPC', 'Item', 'Products', 'Facts', 'FY 2020', 'FY 2019',
       'YTD 2021', 'YTD 2020', 'Latest 13 Weeks', 'Latest 26 Weeks', 'Q4 2019',
       'Q3 2019', 'Q2 2019', '4 W/E 02/27/21'],
      dtype='object')

In [26]:
Kris_LHS.rename(columns={'Markets':"MKT", 'Item':"PROD", 'Manufacturer':"CP.MANUFACTURER",
                           'Franchise':"CP.FRANCHISE", 'Brand':"CP.BRAND",
                           'SubBrand':"CP.SUBBRAND", 'UPC':"PRODUCT.CODE",
                           },inplace=True)

In [27]:
df1=pd.melt(Kris_LHS, id_vars =["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [28]:
df2=df1.pivot_table(index=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [29]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [30]:
df2["Prcnt_ACV"]=df2["Store_selling"]

final_column_sorting=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg",
                      "Prcnt_ACV","Units","Units_Velocity"
                      ]
Kris_Data_final=df2[final_column_sorting]

In [31]:
Kris_Data_final.head(3)

,MKT,PROD,CP.MANUFACTURER,CP.FRANCHISE,CP.BRAND,CP.SUBBRAND,PRODUCT.CODE,PER,Doll_Shr,Doll_Shr_Chg,...,Store_selling_Chg,Velocity,Velocity_Chg,Cum_Distri_Pts,Cum_Distri_Pts_Chg,Dollars,Dollars_Chg,Prcnt_ACV,Units,Units_Velocity
0,Ahold,SOFTSOAP HAND SOAP HAND LIQUID BLACK RASPBERRY...,COLGATE-PALMOLIVE,SOFTSOAP,SFTSP BETTER,SOFTSOAP BETTER FRAGRANCE,007418244575,4 W/E 02/27/21,0.0,-0.463,...,-33.823,0.000,-9.848,0.000,-33.823,0.00,-100.000,0.000,0.0,0.000
2,Ahold,SOFTSOAP HAND SOAP HAND LIQUID BLACK RASPBERRY...,COLGATE-PALMOLIVE,SOFTSOAP,SFTSP BETTER,SOFTSOAP BETTER FRAGRANCE,007418244575,Latest 13 Weeks,0.0,-0.580,...,-37.267,5.071,-5.165,0.007,-37.267,2.49,-99.991,0.007,1.0,2.037
3,Ahold,SOFTSOAP HAND SOAP HAND LIQUID BLACK RASPBERRY...,COLGATE-PALMOLIVE,SOFTSOAP,SFTSP BETTER,SOFTSOAP BETTER FRAGRANCE,007418244575,Latest 26 Weeks,0.0,-0.554,...,-40.361,4.361,-4.690,0.039,-40.361,24.90,-99.953,0.039,10.0,1.751


In [32]:
path_LHS = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/LHS/"

Kris_Data_final.to_csv(path_LHS+"kris_data_LHS.csv",index=False)

## LDL

In [68]:
path_LDL = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/LDL/"

In [73]:
Kris_LDL=Kris_Data[Kris_Data["ShareBasis"]=="LDL/DISHAIDS/WPS/FM/SPRAYS"]

In [56]:
# Facts=["Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg","Prcnt_ACV","Units","Units_Velocity"]
# Markets=[]

In [74]:
Kris_LDL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10850 entries, 0 to 10849
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Markets          10850 non-null  object 
 1   ShareBasis       10850 non-null  object 
 2   Manufacturer     10850 non-null  object 
 3   Franchise        10850 non-null  object 
 4   Brand            10850 non-null  object 
 5   SubBrand         10850 non-null  object 
 6   UPC              10850 non-null  object 
 7   Item             10850 non-null  object 
 8   Products         10850 non-null  object 
 9   Facts            10850 non-null  object 
 10  FY 2020          10850 non-null  float64
 11  FY 2019          10850 non-null  float64
 12  FY 2018          10850 non-null  float64
 13  FY 2017          10850 non-null  float64
 14  YTD 2021         10850 non-null  float64
 15  YTD 2020         10850 non-null  float64
 16  Latest 13 Weeks  10850 non-null  float64
 17  Latest 26 We

In [75]:
Time_Period=list(Kris_LDL.columns[21:22])+list(Kris_LDL.columns[16:18])+list(Kris_LDL.columns[14:15])+list(Kris_LDL.columns[10:11])
print(Time_Period)

['4 W/E 02/27/21', 'Latest 13 Weeks', 'Latest 26 Weeks', 'YTD 2021', 'FY 2020']


In [76]:
set(Kris_LDL["Facts"])

{'Base_Doll_Shr',
 'Base_Doll_Shr_Chg',
 'Cum_Distri_Pts',
 'Cum_Distri_Pts_Chg',
 'Doll_Shr',
 'Doll_Shr_Chg',
 'Dollars',
 'Dollars_Chg',
 'Store_selling',
 'Store_selling_Chg',
 'Units',
 'Units_Velocity',
 'Velocity',
 'Velocity_Chg'}

In [77]:
Kris_LDL.columns

Index(['Markets', 'ShareBasis', 'Manufacturer', 'Franchise', 'Brand',
       'SubBrand', 'UPC', 'Item', 'Products', 'Facts', 'FY 2020', 'FY 2019',
       'FY 2018', 'FY 2017', 'YTD 2021', 'YTD 2020', 'Latest 13 Weeks',
       'Latest 26 Weeks', 'Q4 2019', 'Q3 2019', 'Q2 2019', '4 W/E 02/27/21'],
      dtype='object')

In [78]:
Kris_LDL.rename(columns={'Markets':"MKT", 'Item':"PROD", 'Manufacturer':"CP.MANUFACTURER",
                           'Franchise':"CP.FRANCHISE", 'Brand':"CP.BRAND","ShareBasis":"CP_SHARE_BASIS",
                           'SubBrand':"CP.SUBBRAND", 'UPC':"PRODUCT.CODE",
                           },inplace=True)

In [79]:
df1=pd.melt(Kris_LDL, id_vars =["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","Facts","CP_SHARE_BASIS"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [80]:
df2=df1.pivot_table(index=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE",'PER',"CP_SHARE_BASIS"],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [81]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [82]:
df2["Prcnt_ACV"]=df2["Store_selling"]

final_column_sorting=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg",
                      "Prcnt_ACV","Units","Units_Velocity"
                      ]
Kris_Data_final=df2[final_column_sorting]

In [83]:
Kris_Data_final.head(3)

,MKT,PROD,CP.MANUFACTURER,CP.FRANCHISE,CP.BRAND,CP.SUBBRAND,PRODUCT.CODE,PER,Doll_Shr,Doll_Shr_Chg,...,Store_selling_Chg,Velocity,Velocity_Chg,Cum_Distri_Pts,Cum_Distri_Pts_Chg,Dollars,Dollars_Chg,Prcnt_ACV,Units,Units_Velocity
0,Ahold,AJAX DISHWASHING DETERGENT LIQUID FRESH LEMON ...,COLGATE-PALMOLIVE,AJAX,AJAX,AJAX SUPER DEGREASER,003500049861,4 W/E 02/27/21,0.838,0.198,...,12.355,19.905,-0.668,47.343,12.355,21374.65,32.096,47.343,6462.0,6.018
2,Ahold,AJAX DISHWASHING DETERGENT LIQUID FRESH LEMON ...,COLGATE-PALMOLIVE,AJAX,AJAX,AJAX SUPER DEGREASER,003500049861,Latest 13 Weeks,0.535,-0.184,...,-3.594,19.488,-3.115,32.089,-3.594,46096.50,-21.508,32.089,13940.0,5.893
3,Ahold,AJAX DISHWASHING DETERGENT LIQUID FRESH LEMON ...,COLGATE-PALMOLIVE,AJAX,AJAX,AJAX SUPER DEGREASER,003500049861,Latest 26 Weeks,0.373,-0.340,...,-12.571,19.320,-2.357,22.781,-12.571,64786.18,-41.828,22.781,19673.0,5.867


In [84]:
Kris_Data_final.to_csv(path_LDL+"kris_data_LDL.csv",index=False)

## FS

In [ ]:
path_FS = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/FS/"

In [ ]:
Kris_FS=Kris_Data[Kris_Data["ShareBasis"].isin(['LIQUID FS', 'SCENT BOOSTERS', 'SHEETS'])]

In [ ]:
# Facts=["Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg","Prcnt_ACV","Units","Units_Velocity"]
# Markets=[]

In [ ]:
set(Kris_FS["ShareBasis"])

In [ ]:
Kris_FS.info()

In [ ]:
Time_Period=list(Kris_FS.columns[21:22])+list(Kris_FS.columns[16:18])+list(Kris_FS.columns[14:15])+list(Kris_FS.columns[10:11])
print(Time_Period)

In [ ]:
Kris_FS.columns

In [ ]:
Kris_FS.rename(columns={'Markets':"MKT", 'Item':"PROD", 'Manufacturer':"CP.MANUFACTURER",
                           'Franchise':"CP.FRANCHISE", 'Brand':"CP.BRAND","ShareBasis":"CP_SHARE_BASIS",
                           'SubBrand':"CP.SUBBRAND", 'UPC':"PRODUCT.CODE",
                           },inplace=True)

In [ ]:
df1=pd.melt(Kris_FS, id_vars =["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","Facts","CP_SHARE_BASIS"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE",'PER',"CP_SHARE_BASIS"],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
df2["Prcnt_ACV"]=df2["Store_selling"]

final_column_sorting=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg",
                      "Prcnt_ACV","Units","Units_Velocity","CP_SHARE_BASIS"
                      ]
Kris_Data_final=df2[final_column_sorting]

In [ ]:
Kris_Data_final.head(3)

In [ ]:
Kris_Data_final.to_csv(path_FS+"kris_data_FS.csv",index=False)

## BW

In [ ]:
path_BW = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/BW/"

In [ ]:
Kris_BW=Kris_Data[Kris_Data["ShareBasis"]=="BODYWASH"]

In [ ]:
# Facts=["Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg","Prcnt_ACV","Units","Units_Velocity"]
# Markets=[]

In [ ]:
Kris_BW.info()

In [ ]:
Time_Period=list(Kris_BW.columns[19:20])+list(Kris_BW.columns[14:16])+list(Kris_BW.columns[12:13])+list(Kris_BW.columns[10:11])
print(Time_Period)

In [ ]:
Kris_BW.columns

In [ ]:
Kris_BW.rename(columns={'Markets':"MKT", 'Item':"PROD", 'Manufacturer':"CP.MANUFACTURER",
                           'Franchise':"CP.FRANCHISE", 'Brand':"CP.BRAND",
                           'SubBrand':"CP.SUBBRAND", 'UPC':"PRODUCT.CODE",
                           },inplace=True)

In [ ]:
df1=pd.melt(Kris_BW, id_vars =["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
df2["Prcnt_ACV"]=df2["Store_selling"]

final_column_sorting=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg",
                      "Prcnt_ACV","Units","Units_Velocity"
                      ]
Kris_Data_final=df2[final_column_sorting]

In [ ]:
Kris_Data_final.head(6)

In [ ]:
Kris_Data_final.to_csv(path_BW+"kris_data_BW.csv",index=False)

### BAR

In [ ]:
Kris_BAR=Kris_Data[Kris_Data["ShareBasis"]=="BAR"]

In [ ]:
# Facts=["Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg","Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg","Prcnt_ACV","Units","Units_Velocity"]
# Markets=[]

In [ ]:
Kris_BAR.info()

In [ ]:
Time_Period=list(Kris_BAR.columns[19:20])+list(Kris_BAR.columns[14:16])+list(Kris_BAR.columns[12:13])+list(Kris_BAR.columns[10:11])
print(Time_Period)

In [ ]:
Kris_BAR.columns

In [ ]:
Kris_BAR.rename(columns={'Markets':"MKT", 'Item':"PROD", 'Manufacturer':"CP.MANUFACTURER",
                           'Franchise':"CP.FRANCHISE", 'Brand':"CP.BRAND",
                           'SubBrand':"CP.SUBBRAND", 'UPC':"PRODUCT.CODE",
                           },inplace=True)

In [ ]:
df1=pd.melt(Kris_BAR, id_vars =["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","Facts"], value_vars =Time_Period,var_name="PER")

df1["value"]=df1["value"].apply(pd.to_numeric)

In [ ]:
df2=df1.pivot_table(index=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE",'PER'],columns='Facts')

df2.columns=df2.columns.droplevel().rename(None)
df2.reset_index(inplace =True)

In [ ]:
from pandas.api.types import CategoricalDtype

Time_Period_order = CategoricalDtype(
    Time_Period, 
    ordered=True
)

df2['PER'] = df2['PER'].astype(Time_Period_order)

df2.sort_values(['MKT','PROD','PER'],inplace=True)

In [ ]:
df2["Prcnt_ACV"]=df2["Store_selling"]

final_column_sorting=["MKT","PROD","CP.MANUFACTURER","CP.FRANCHISE","CP.BRAND","CP.SUBBRAND","PRODUCT.CODE","PER",
                      "Doll_Shr","Doll_Shr_Chg","Base_Doll_Shr","Base_Doll_Shr_Chg","Store_selling","Store_selling_Chg",
                      "Velocity","Velocity_Chg","Cum_Distri_Pts","Cum_Distri_Pts_Chg","Dollars","Dollars_Chg",
                      "Prcnt_ACV","Units","Units_Velocity"
                      ]
Kris_Data_final=df2[final_column_sorting]

In [ ]:
Kris_Data_final.head(15)

In [ ]:
Kris_BAR = "//falmumapp34/Nielsen Connect Transition & Maintenance (20-SCP-3650)/05. Output Data/5P Output/BAR/"

Kris_Data_final.to_csv(Kris_BAR+"kris_data_BAR.csv",index=False)